# Property Investment Metric Analysis

This notebook loads the pre-processed property investment dataset and performs additional metric analysis.

In [4]:
# Import required libraries
import pandas as pd
import numpy as np
import os

print("📦 Libraries imported successfully!")

📦 Libraries imported successfully!


In [ ]:
# Load the property investment dataset from pickle file
print("=== LOADING PROPERTY INVESTMENT DATASET ===")
print()

# Check if the file exists
pkl_file_path = 'Dataset.pkl'

if os.path.exists(pkl_file_path):
    # Load the dataset
    df = pd.read_pickle(pkl_file_path)
    
    print(f"✅ Dataset loaded successfully from: {pkl_file_path}")
    print(f"   • Shape: {df.shape}")
    print(f"   • File size: {os.path.getsize(pkl_file_path) / 1024:.1f} KB")
    print()
    
    print("📊 DATASET OVERVIEW:")
    print(f"   • Total property parcels: {len(df):,}")
    print(f"   • Total columns: {len(df.columns)}")
    print(f"   • Geographic regions: {df['sa4'].nunique()} unique SA4 areas")
    print(f"   • States covered: {', '.join(df['state'].unique())}")
    
    if 'comprehensive_score' in df.columns:
        print(f"   • Comprehensive score range: {df['comprehensive_score'].min():.1f} - {df['comprehensive_score'].max():.1f}")
        print(f"   • Average comprehensive score: {df['comprehensive_score'].mean():.1f}")
    
    print()
    print("📋 AVAILABLE COLUMNS:")
    for i, col in enumerate(df.columns, 1):
        print(f"   {i:2d}. {col}")
        
else:
    print(f"❌ Error: File not found at {pkl_file_path}")
    print("Please make sure you've run the data preparation notebook first to generate the dataset.")
    print()
    print("Expected file location: output/property_investment_analysis_results.pkl")

=== LOADING PROPERTY INVESTMENT DATASET ===

❌ Error: File not found at property_investment_analysis_results.pkl
Please make sure you've run the data preparation notebook first to generate the dataset.

Expected file location: output/property_investment_analysis_results.pkl


In [6]:
# Display sample data and basic statistics
print("=== DATASET SAMPLE & STATISTICS ===")
print()

if 'df' in locals():
    print("📋 SAMPLE DATA (First 5 rows):")
    display_columns = ['sa4', 'area_hectares', 'comprehensive_score', 'comprehensive_grade', 
                      'accessibility_score', 'market_activity_score', 'property_value_score']
    available_display_cols = [col for col in display_columns if col in df.columns]
    print(df[available_display_cols].head())
    print()
    
    print("📈 KEY STATISTICS:")
    if 'comprehensive_score' in df.columns:
        print(f"Comprehensive Score Statistics:")
        print(df['comprehensive_score'].describe().round(2))
        print()
    
    if 'comprehensive_grade' in df.columns:
        print("Investment Grade Distribution:")
        print(df['comprehensive_grade'].value_counts().sort_index())
        print()
    
    print("🎯 READY FOR METRIC ANALYSIS!")
    print("The dataset is now loaded and ready for additional property investment metric calculations.")
    
else:
    print("⚠️  Dataset not loaded. Please run the previous cell first.")

=== DATASET SAMPLE & STATISTICS ===

⚠️  Dataset not loaded. Please run the previous cell first.
